In [ ]:
import numpy as np
import pandas as pd
from math import sqrt
import xgboost as xgb
import lightgbm as lgb
from scipy import stats
import tensorflow as tf
from tensorflow import keras
import sklearn.linear_model as lm
from tensorflow.keras import layers
import statsmodels.formula.api as smf
from sklearn.preprocessing import scale
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tools.tools import add_constant
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
#Day 1

In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
df.head()

,trip_duration,distance_traveled,num_of_passengers,fare,tip,miscellaneous_fees,total_fare,surge_applied
0,748.0,2.75,1.0,75.00,24,6.300,105.300,0
1,1187.0,3.43,1.0,105.00,24,13.200,142.200,0
2,730.0,3.12,1.0,71.25,0,26.625,97.875,1
3,671.0,5.63,3.0,90.00,0,9.750,99.750,0
4,329.0,2.09,1.0,45.00,12,13.200,70.200,0


In [ ]:
df.isna().sum()

trip_duration         0
distance_traveled     0
num_of_passengers     0
fare                  0
tip                   0
miscellaneous_fees    0
total_fare            0
surge_applied         0
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209673 entries, 0 to 209672
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   trip_duration       209673 non-null  float64
 1   distance_traveled   209673 non-null  float64
 2   num_of_passengers   209673 non-null  float64
 3   fare                209673 non-null  float64
 4   tip                 209673 non-null  int64  
 5   miscellaneous_fees  209673 non-null  float64
 6   total_fare          209673 non-null  float64
 7   surge_applied       209673 non-null  int64  
dtypes: float64(6), int64(2)
memory usage: 12.8 MB


In [ ]:
col = df.columns
for c in col:
  l = df[c].unique()
  print(c,len(l))

trip_duration 5907
distance_traveled 2463
num_of_passengers 10
fare 631
tip 262
miscellaneous_fees 1674
total_fare 3742
surge_applied 2


In [ ]:
df['num_of_passengers'] = df['num_of_passengers'].astype(str)
df['surge_applied'] = df['surge_applied'].astype(str)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209673 entries, 0 to 209672
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   trip_duration       209673 non-null  float64
 1   distance_traveled   209673 non-null  float64
 2   num_of_passengers   209673 non-null  object 
 3   fare                209673 non-null  float64
 4   tip                 209673 non-null  int64  
 5   miscellaneous_fees  209673 non-null  float64
 6   total_fare          209673 non-null  float64
 7   surge_applied       209673 non-null  object 
dtypes: float64(5), int64(1), object(2)
memory usage: 12.8+ MB


In [ ]:
df

,trip_duration,distance_traveled,num_of_passengers,fare,tip,miscellaneous_fees,total_fare,surge_applied
0,748.0,2.75,1.0,75.00,24,6.300,105.300,0
1,1187.0,3.43,1.0,105.00,24,13.200,142.200,0
2,730.0,3.12,1.0,71.25,0,26.625,97.875,1
3,671.0,5.63,3.0,90.00,0,9.750,99.750,0
4,329.0,2.09,1.0,45.00,12,13.200,70.200,0
...,...,...,...,...,...,...,...,...
209668,1617.0,8.42,1.0,150.00,47,5.800,202.800,0
209669,438.0,1.29,1.0,48.75,12,34.575,95.325,1
209670,571.0,2.82,1.0,63.75,0,6.000,69.750,0
209671,491.0,2.16,1.0,56.25,0,13.500,69.750,0


In [ ]:
df[['trip_durationS','distance_traveledS','fareS','tipS','miscellaneous_feesS']] = scale(df[['trip_duration','distance_traveled','fare','tip','miscellaneous_fees']],with_mean=True,with_std=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209673 entries, 0 to 209672
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   trip_duration        209673 non-null  float64
 1   distance_traveled    209673 non-null  float64
 2   num_of_passengers    209673 non-null  object 
 3   fare                 209673 non-null  float64
 4   tip                  209673 non-null  int64  
 5   miscellaneous_fees   209673 non-null  float64
 6   total_fare           209673 non-null  float64
 7   surge_applied        209673 non-null  object 
 8   trip_durationS       209673 non-null  float64
 9   distance_traveledS   209673 non-null  float64
 10  fareS                209673 non-null  float64
 11  tipS                 209673 non-null  float64
 12  miscellaneous_feesS  209673 non-null  float64
dtypes: float64(10), int64(1), object(2)
memory usage: 20.8+ MB


In [ ]:
df2 = df[['trip_durationS','distance_traveledS','fareS','tipS','miscellaneous_feesS']]
df3 = add_constant(df2)

vif_df = pd.DataFrame()
vif_df["feature"] = df3.columns
print(vif_df)

vif_df["VIF"] = [variance_inflation_factor(df3.values,i) for i in range(len(df3.columns))]
print(vif_df)

               feature
0                const
1       trip_durationS
2   distance_traveledS
3                fareS
4                 tipS
5  miscellaneous_feesS
               feature       VIF
0                const  1.000000
1       trip_durationS  1.023465
2   distance_traveledS  1.001426
3                fareS  1.184016
4                 tipS  1.176302
5  miscellaneous_feesS  1.173985


In [ ]:
fit1 = smf.ols('total_fare~trip_durationS+distance_traveledS+fareS+tipS+miscellaneous_feesS+num_of_passengers+surge_applied',data=df).fit()
fit1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             total_fare   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 4.190e+33
Date:                Mon, 12 Jun 2023   Prob (F-statistic):               0.00
Time:                        22:30:27   Log-Likelihood:             5.8550e+06
No. Observations:              209673   AIC:                        -1.171e+07
Df Residuals:                  209657   BIC:                        -1.171e+07
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                  127.7911   6.59e-15   1.94e+16      0.000     127.791     127.791
num_of_passengers[T.1.0]  3.341e-13   6.59e-15     50.698      0.000    3.21e-13    3.47e-13
num_of_passengers[T.2.0]  2.282e-13   6.72e-15     33.976      0.000    2.15e-13    2.41e-13
num_of_passengers[T.3.0]  3.733e-13   7.42e-15     50.292      0.000    3.59e-13    3.88e-13
num_of_passengers[T.4.0]  9.974e-14    8.9e-15     11.212      0.000    8.23e-14    1.17e-13
num_of_passengers[T.5.0]  1.361e-13   7.08e-15     19.217      0.000    1.22e-13     1.5e-13
num_of_passengers[T.6.0]  3.499e-13   7.27e-15     48.124      0.000    3.36e-13    3.64e-13
num_of_passengers[T.7.0] -9.795e-13   1.28e-13     -7.666      0.000   -1.23e-12   -7.29e-13
num_of_passengers[T.8.0]  1.218e-12   3.67e-14     33.211      0.000    1.15e-12    1.29e-12
num_of_passengers[T.9.0]  2.495e-12   6.41e-14     38.892      0.000    2.37e-12    2.62e-12
surge_applied[T.1]       -1.704e-14   1.26e-15    -13.483      0.000   -1.95e-14   -1.46e-14
trip_durationS             1.97e-14   3.99e-16     49.418      0.000    1.89e-14    2.05e-14
distance_traveledS       -3.306e-14   3.94e-16    -83.814      0.000   -3.38e-14   -3.23e-14
fareS                       85.6025   4.32e-16   1.98e+17      0.000      85.602      85.602
tipS                        20.3677   4.28e-16   4.76e+16      0.000      20.368      20.368
miscellaneous_feesS         12.5534   5.95e-16   2.11e+16      0.000      12.553      12.553
==============================================================================
Omnibus:                   177846.673   Durbin-Watson:                   1.156
Prob(Omnibus):                  0.000   Jarque-Bera (JB):       4872673963.237
Skew:                           2.234   Prob(JB):                         0.00
Kurtosis:                     749.810   Cond. No.                         446.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
fit2 = smf.ols('total_fare~trip_duration+distance_traveled+fare+tip+miscellaneous_fees+num_of_passengers+surge_applied',data=df).fit()
fit2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             total_fare   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 7.504e+32
Date:                Mon, 12 Jun 2023   Prob (F-statistic):               0.00
Time:                        22:30:29   Log-Likelihood:             5.6747e+06
No. Observations:              209673   AIC:                        -1.135e+07
Df Residuals:                  209657   BIC:                        -1.135e+07
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                 2.755e-13   1.56e-14     17.656      0.000    2.45e-13    3.06e-13
num_of_passengers[T.1.0] -1.752e-13   1.56e-14    -11.252      0.000   -2.06e-13   -1.45e-13
num_of_passengers[T.2.0]  2.079e-14   1.59e-14      1.310      0.190   -1.03e-14    5.19e-14
num_of_passengers[T.3.0]  2.544e-13   1.75e-14     14.509      0.000     2.2e-13    2.89e-13
num_of_passengers[T.4.0]  1.503e-13    2.1e-14      7.152      0.000    1.09e-13    1.92e-13
num_of_passengers[T.5.0]  2.111e-13   1.67e-14     12.613      0.000    1.78e-13    2.44e-13
num_of_passengers[T.6.0]  5.326e-13   1.72e-14     30.997      0.000    4.99e-13    5.66e-13
num_of_passengers[T.7.0] -9.455e-13   3.02e-13     -3.132      0.002   -1.54e-12   -3.54e-13
num_of_passengers[T.8.0]   6.89e-13   8.67e-14      7.948      0.000    5.19e-13    8.59e-13
num_of_passengers[T.9.0]  2.716e-12   1.52e-13     17.923      0.000    2.42e-12    3.01e-12
surge_applied[T.1]       -8.501e-15   2.99e-15     -2.847      0.004   -1.44e-14   -2.65e-15
trip_duration            -8.417e-17   1.97e-19   -426.669      0.000   -8.46e-17   -8.38e-17
distance_traveled         -3.32e-16   7.44e-18    -44.605      0.000   -3.47e-16   -3.17e-16
fare                         1.0000   1.19e-17   8.38e+16      0.000       1.000       1.000
tip                          1.0000   4.96e-17   2.01e+16      0.000       1.000       1.000
miscellaneous_fees           1.0000   1.12e-16   8.93e+15      0.000       1.000       1.000
==============================================================================
Omnibus:                   388507.272   Durbin-Watson:                   1.976
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        514101109.537
Skew:                          14.205   Prob(JB):                         0.00
Kurtosis:                     243.913   Cond. No.                     1.59e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.59e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
x = df[['trip_durationS','distance_traveledS','fareS','tipS','miscellaneous_feesS','num_of_passengers','surge_applied']]
y = df['total_fare']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=1)

In [ ]:
modelc = lm.LinearRegression()
modelc.fit(x,y)

LinearRegression()

In [ ]:
pred = modelc.predict(x_test)
print(mean_squared_error(y_test,pred))

1.810365530580223e-27


In [ ]:
x = df[['trip_duration','distance_traveled','fare','tip','miscellaneous_fees','num_of_passengers','surge_applied']]
y = df['total_fare']
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=1)
modelc = lm.LinearRegression()
modelc.fit(x,y)
pred = modelc.predict(x_test)
print(mean_squared_error(y_test,pred))

2.695498023897242e-26


In [ ]:
#Day 2

In [ ]:
#LightGBM
train_df = pd.read_csv('train.csv')

train_size = int(0.8 * len(train_df))
train_data = train_df[:train_size]
test_data = train_df[train_size:]

X_train = train_data.drop(['total_fare', 'fare'], axis=1)
y_train = train_data['fare']
X_test = test_data.drop(['total_fare', 'fare'], axis=1)
y_test = test_data['fare']

params = {
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 100,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

model = lgb.LGBMRegressor(**params)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

rmse = sqrt(mean_squared_error(y_test, y_pred))
print('RMSE:', rmse)

test_df = pd.read_csv('test.csv')
X_final_test = test_df.drop(['total_fare', 'fare'], axis=1)
fare_predictions = model.predict(X_final_test)
test_df['fare'] = fare_predictions

print(test_df)

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015649 seconds.
You can set `force_col_wise=true` to remove the overhead.
RMSE: 34.440733188221905
       trip_duration  distance_traveled  num_of_passengers        fare  tip  \
0             1076.0             

In [ ]:
df = pd.read_csv('train.csv')

X = df[['trip_duration', 'distance_traveled', 'num_of_passengers', 'tip', 'miscellaneous_fees', 'surge_applied']]
y = df['fare']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#Neural Network
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = keras.Sequential([
    layers.Dense(256, activation='relu', input_shape=[X_train_scaled.shape[1]]),
    layers.Dense(128, activation='relu'),
    layers.Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train_scaled, y_train, epochs=12, batch_size=50)
y_pred = model.predict(X_test_scaled)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
y_pred = abs(y_pred)
rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred))

print(f'RMSE: {rmse}')
print(f'RMSLE: {rmsle}')

Epoch 1/12
3355/3355 [==============================] - 13s 3ms/step - loss: 6340.6729
Epoch 2/12
3355/3355 [==============================] - 9s 3ms/step - loss: 4110.1758
Epoch 3/12
3355/3355 [==============================] - 9s 3ms/step - loss: 3157.5671
Epoch 4/12
3355/3355 [==============================] - 9s 3ms/step - loss: 3082.1963
Epoch 5/12
3355/3355 [==============================] - 9s 3ms/step - loss: 3047.3481
Epoch 6/12
3355/3355 [==============================] - 8s 2ms/step - loss: 3023.3982
Epoch 7/12
3355/3355 [==============================] - 9s 3ms/step - loss: 3007.7002
Epoch 8/12
3355/3355 [==============================] - 9s 3ms/step - loss: 2987.6050
Epoch 9/12
3355/3355 [==============================] - 8s 3ms/step - loss: 2965.2456
Epoch 10/12
3355/3355 [==============================] - 9s 3ms/step - loss: 2947.7883
Epoch 11/12
3355/3355 [==============================] - 9s 3ms/step - loss: 2935.6787
Epoch 12/12
1311/1311 [============================

In [ ]:
#Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred))

print(f'RMSE: {rmse}')
print(f'RMSLE: {rmsle}')

RMSE: 39.569807642150714
RMSLE: 0.18738600462217608


In [ ]:
#Random Forest
train_data = pd.concat([X_train, y_train], axis=1)
numerical_columns = ['trip_duration', 'distance_traveled', 'num_of_passengers', 'tip', 'miscellaneous_fees', 'surge_applied']
z_scores = np.abs(stats.zscore(train_data[numerical_columns]))
threshold = 3
outlier_indices = np.where(z_scores > threshold)[0]
train_data_cleaned = train_data.drop(train_data.index[outlier_indices])

X_train_cleaned = train_data_cleaned.drop('fare', axis=1)
y_train_cleaned = train_data_cleaned['fare']

model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train_cleaned, y_train_cleaned)
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred))

print(f'RMSE: {rmse}')
print(f'RMSLE: {rmsle}')

RMSE: 44.224212101846
RMSLE: 0.19380615157482822


In [ ]:
#XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test)

params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse'
}

model = xgb.train(params, dtrain)
y_pred = model.predict(dtest)
y_pred = abs(y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred))

print(f'RMSE: {rmse}')
print(f'RMSLE: {rmsle}')

RMSE: 40.01953201846862
RMSLE: 0.1905309348343421


In [ ]:
#LightGBM
train_data = lgb.Dataset(X_train, label=y_train)

params = {
    'objective': 'regression',
    'metric': 'rmsle',
}

model = lgb.train(params, train_data)
y_pred = model.predict(X_test)
y_pred = abs(y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred))

print(f'RMSE: {rmse}')
print(f'RMSLE: {rmsle}')

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007926 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 959
[LightGBM] [Info] Number of data points in the train set: 167738, number of used features: 6
[LightGBM] [Info] Start training from score 99.493813
RMSE: 38.584179647457425
RMSLE: 0.18200248864364424


In [ ]:
#LightGBM
train_data = pd.concat([X_train, y_train], axis=1)
numerical_columns = ['trip_duration', 'distance_traveled', 'num_of_passengers', 'tip', 'miscellaneous_fees', 'surge_applied']
z_scores = np.abs(stats.zscore(train_data[numerical_columns]))
threshold = 1
outlier_indices = np.where(z_scores > threshold)[0]

train_data_cleaned = train_data.drop(train_data.index[outlier_indices])

X_train_cleaned = train_data_cleaned.drop('fare', axis=1)
y_train_cleaned = train_data_cleaned['fare']

train_data_cleaned_lgb = lgb.Dataset(X_train_cleaned, label=y_train_cleaned)


params = {
    'objective': 'regression',
    'metric': 'rmse',
}

model = lgb.train(params, train_data_cleaned_lgb)
y_pred = model.predict(X_test)
y_pred = abs(y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred))

print(f'RMSE: {rmse}')
print(f'RMSLE: {rmsle}')

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011882 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 731
[LightGBM] [Info] Number of data points in the train set: 101488, number of used features: 5
[LightGBM] [Info] Start training from score 78.547397
RMSE: 47.21635677393502
RMSLE: 0.21241074594167944


In [ ]:
train_df = pd.read_csv('train.csv')

train_size = int(0.8 * len(train_df))
train_data = train_df[:train_size]
test_data = train_df[train_size:]

X_train = train_data.drop(['total_fare', 'fare'], axis=1)
y_train = train_data['fare']
X_test = test_data.drop(['total_fare', 'fare'], axis=1)  # Adjust features here
y_test = test_data['fare']

In [ ]:
#LightGBM
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 100,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

model = lgb.LGBMRegressor(**params)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

rmse = sqrt(mean_squared_error(y_test, y_pred))
print('RMSE:', rmse)

test_df = pd.read_csv('test.csv')
X_final_test = test_df.drop(['total_fare', 'fare'], axis=1)
fare_predictions = model.predict(X_final_test)
test_df['fare'] = fare_predictions

print(test_df)

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014836 seconds.
You can set `force_col_wise=true` to remove the overhead.
RMSE: 34.440733188221905
       trip_duration  distance_traveled  num_of_passengers        fare  tip  \
0             1076.0             

In [ ]:
#Day 3

In [ ]:
#Multiple Models
models = {
    'LightGBM': lgb.LGBMRegressor(),
    'XGBoost': xgb.XGBRegressor(),
    'Random Forest': RandomForestRegressor()
}

results = {}

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    rmse = sqrt(mean_squared_error(y_test, y_pred))

    results[model_name] = {'RMSE': rmse}

best_model = min(results, key=lambda x: results[x]['RMSE'])
print('Best Model:', best_model)
print('RMSE:', results[best_model]['RMSE'])

Best Model: LightGBM
RMSE: 34.079814538003745


In [ ]:
#Moving forward with LightGBM
models = {
    'LightGBM': lgb.LGBMRegressor(reg_alpha = 2,reg_lambda = 0.5),
    # 'XGBoost': xgb.XGBRegressor(),
    # 'Random Forest': RandomForestRegressor()
}

results = {}

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred = abs(y_pred)
    rmse = sqrt(mean_squared_error(y_test, y_pred))

    results[model_name] = {'RMSE': rmse}

best_model = min(results, key=lambda x: results[x]['RMSE'])
print('Best Model:', best_model)
print('RMSE:', results[best_model]['RMSE'])

Best Model: LightGBM
RMSE: 33.92262842364048


In [ ]:
test_df = pd.read_csv('test.csv')
X_final_test = test_df.drop(['total_fare', 'fare'], axis=1)

best_model = models[best_model]
best_model.fit(X_train, y_train)
fare_predictions = best_model.predict(X_final_test)

fare_predictions = abs(fare_predictions)
test_df['fare'] = fare_predictions

In [ ]:
test_df

,trip_duration,distance_traveled,num_of_passengers,fare,tip,miscellaneous_fees,total_fare,surge_applied
0,1076.0,4.18,1.0,97.735307,0,13.500,0,0
1,429.0,1.48,4.0,48.421578,0,13.500,0,0
2,856.0,4.15,1.0,86.644780,24,6.000,0,0
3,622.0,3.22,1.0,69.277001,15,5.625,0,0
4,507.0,3.98,1.0,109.428009,0,2.250,0,0
...,...,...,...,...,...,...,...,...
89856,435.0,2.24,1.0,52.761079,13,13.700,0,0
89857,519.0,2.61,1.0,59.704594,7,13.850,0,0
89858,450.0,2.24,1.0,53.670001,0,26.625,0,1
89859,919.0,4.12,1.0,89.520957,25,30.200,0,1


In [ ]:
test_df['total_fare'] = test_df['fare']+test_df['tip']+test_df['miscellaneous_fees']

In [ ]:
test_df

,trip_duration,distance_traveled,num_of_passengers,fare,tip,miscellaneous_fees,total_fare,surge_applied
0,1076.0,4.18,1.0,97.735307,0,13.500,111.235307,0
1,429.0,1.48,4.0,48.421578,0,13.500,61.921578,0
2,856.0,4.15,1.0,86.644780,24,6.000,116.644780,0
3,622.0,3.22,1.0,69.277001,15,5.625,89.902001,0
4,507.0,3.98,1.0,109.428009,0,2.250,111.678009,0
...,...,...,...,...,...,...,...,...
89856,435.0,2.24,1.0,52.761079,13,13.700,79.461079,0
89857,519.0,2.61,1.0,59.704594,7,13.850,80.554594,0
89858,450.0,2.24,1.0,53.670001,0,26.625,80.295001,1
89859,919.0,4.12,1.0,89.520957,25,30.200,144.720957,1


In [ ]:
final = test_df['total_fare']
final

0        111.235307
1         61.921578
2        116.644780
3         89.902001
4        111.678009
            ...    
89856     79.461079
89857     80.554594
89858     80.295001
89859    144.720957
89860     92.084162
Name: total_fare, Length: 89861, dtype: float64

In [ ]:
final.to_csv('lastpred.csv',index=False)

In [ ]:
#Final Submission